### Reading the Raw File

In [0]:
# df_tracks = spark.read.json("/Volumes/capstone_project/capstone_project_raw/raw_landing/df_tracks_completo.json")
# display(df_tracks)

In [0]:
import pandas as pd
from datetime import datetime

df_raw_tracks = pd.read_json("/Volumes/capstone_project/capstone_project_raw/raw_landing/df_tracks_completo.json")
df_raw_tracks.insert(0, "audit_ts", datetime.now())
display(df_raw_tracks)

In [0]:
df_raw_tracks.info()

### Ingesting to Delta Table in Raw Layer

Estamos usando o modo overwrite por enquanto para sobre escrever.
Em uma situação em produção, onde temos dado novo chegando a cada minuto, temos que usar o modo insert, update or upsert, como o "Merge".
Em produção, o certo também é criar uma tabela com schema definido, com unique_constraints properties.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType

schema = StructType([
    StructField("audit_ts", TimestampType(), True),
    StructField("playlist", StringType(), True),
    StructField("track_id", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("artist", StringType(), True),
    StructField("artist_id", StringType(), True),
    StructField("artist_image", DoubleType(), True),
    StructField("album", StringType(), True),
    StructField("album_image", StringType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True)
])

raw_tracks = spark.createDataFrame(df_raw_tracks, schema=schema)

raw_tracks.write.format("delta").mode("overwrite").saveAsTable("capstone_project.capstone_project_raw.raw_tracks")